In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.svm import LinearSVC

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
train_df=pd.read_csv('C:\\Users\\Admin34\\Desktop\\4k.2C - Copy.csv')
train_df

,Review,Rating
0,The rooms are dirty the curtains were filthy. ...,1
1,I have to say I was nervous after reading some...,3
2,My son had to take my husband's place on the h...,3
3,"This was the 3rd time visiting New York, wante...",5
4,I am an essential worker I work at Sloan-Kette...,5
...,...,...
2994,"We had a great time staying at this hotel, per...",5
2995,Front desk staff were not friendly. Elevators ...,3
2996,For past several years each stay has seen a de...,1
2997,"In short, the freight train sounding air-condi...",1


In [12]:
test_df=pd.read_csv('C:\\Users\\Admin34\\Desktop\\4k.2C - copytest.csv')
test_df

,Review
0,I recently stayed here for work. The hotel has...
1,The only one thing that this hotel offers is i...
2,"Very good location, but that is all I can say ..."
3,So I am writing an unbiased review regarding t...
4,This would have been a 4 star review but I got...
...,...
1019,We stayed at the Roosevelt for seven nights. M...
1020,We stayed at the Roosevelt Hotel expecting an ...
1021,Was a little worried prior to staying here ove...
1022,This was the second time we stayed at The Roos...


In [8]:
import re

In [13]:
#Remove html links and entity references
def html_references(text):
    texts = text  
    # remove url - references to websites
    url_remove  = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    texts  = re.sub(url_remove, '', texts)
    # remove common html entity references in utf-8 as '&lt;', '&gt;', '&amp;'
    entities_remove = r'&amp;|&gt;|&lt'
    texts = re.sub(entities_remove, "", texts)
    # split into words by white space
    words = texts.split()
    #convert to lower case
    words = [word.lower() for word in words]
    return " ".join(words)
train_df['Review'] = train_df['Review'].apply(lambda x : html_references(x))
test_df['Review'] = test_df['Review'].apply(lambda x : html_references(x))

In [14]:
def decontraction(text):
    # specific
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)
    
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    return text 
train_df['Review'] = train_df['Review'].apply(lambda x : decontraction(x))
test_df['Review'] = test_df['Review'].apply(lambda x : decontraction(x))

In [15]:
def separate_alphanumeric(text):
    words = text
    # separate alphanumeric
    words = re.findall(r"[^\W\d_]+|\d+", words)
    return " ".join(words)
train_df['Review'] = train_df['Review'].apply(lambda x : separate_alphanumeric(x))
test_df['Review'] = test_df['Review'].apply(lambda x : separate_alphanumeric(x))

In [16]:
#Change repetitive characters
def cont_rep_char(text):
    tchr = text.group(0) 
    
    if len(tchr) > 1:
        return tchr[0:2] # take max of 2 consecutive letters
def unique_char(rep, tweets):
    substitute = re.sub(r'(\w)\1+', rep, tweets)
    return substitute
train_df['Review'] = (train_df['Review'].astype('str').apply(lambda x : unique_char(cont_rep_char, x)))
test_df['Review'] = (test_df['Review'].astype('str').apply(lambda x : unique_char(cont_rep_char, x)))

In [17]:
#!pip install wordninja

In [18]:
import wordninja # !pip install wordninja
def split_attached_words(text):
    words = wordninja.split(text)
    return" ".join(words)
train_df['Review'] = train_df['Review'].apply(lambda x : split_attached_words(x))
test_df['Review'] = test_df['Review'].apply(lambda x : split_attached_words(x))

In [19]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
#print(stop_words)

In [20]:
def stopwords_shortwords(text):
    # filter out stop words
    words = text.split()
    #stop_words = set(stopwords.words( 'english' ))
    words = [w for w in words if not w in stop_words]
    # filter out short tokens
    for word in words:
        if word.isalpha():
            words = [word for word in words if len(word) > 1 ]
        else:
            words = [word for word in words]
    return" ".join(words)
train_df['Review'] = train_df['Review'].apply(lambda x : stopwords_shortwords(x))
test_df['Review'] = test_df['Review'].apply(lambda x : stopwords_shortwords(x))

In [21]:
df=pd.concat([train_df,test_df])
df.shape

(4023, 2)

In [22]:
#!pip install pyspellchecker

In [23]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [24]:
df['Review']=df['Review'].apply(lambda x : correct_spellings(x))

In [25]:
def changeRating(rating):
    if rating==1 or rating==2:
        return 1
    elif rating==4 or rating==5 :
        return 3
        
    else:
        return 2

In [26]:
df["New_Rating"]=df["Rating"].apply(changeRating)
df.head()

,Review,Rating,New_Rating
0,rooms dirty curtains filthy totally priced tou...,1.0,1
1,say nervous reading reviews regarding hotel mi...,3.0,2
2,son take husband place holiday immediately bc ...,3.0,2
3,rd time visiting new york wanted older grand h...,5.0,3
4,essential worker work sloan kettering ms cc ho...,5.0,3


In [27]:
train1=df[:train_df.shape[0]]
test1=df[train_df.shape[0]:]

In [28]:
X=train1['Review']
y=train1['New_Rating']

In [29]:
#split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2249,)
(750,)
(2249,)
(750,)


In [31]:
from nltk.tokenize import word_tokenize

In [33]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize, lowercase=False, ngram_range=(1,5))

In [34]:
#transforming test data and train data in to a document term matrix
X_Train_Dtm=vectorizer.fit_transform(X_train)
X_Test_Dtm=vectorizer.transform(X_test)

In [35]:
type(X_test)

pandas.core.series.Series

In [36]:
X_test

2056    stayed st patrick day weekend roosevelt old ho...
1742    hotel lobby gorgeous feel like scene great gat...
2578    pleasure stay hotel recent conference nec easy...
2100    stayed nights hotel lobby bar looked stunning ...
1787    average hotel best gripes found reviews person...
                              ...                        
257     excellent hotel stayed family reunion location...
2544    hotel choice nec stay roosevelt least 10 nts y...
1730    good location pricey restaurant convenience st...
2359    location perfect midtown close everything near...
216     hotel average best arrive see lobby area bar s...
Name: Review, Length: 750, dtype: object

In [37]:
from imblearn.over_sampling import SMOTE
smote = SMOTE('minority')

X_sm_t, y_sm_t = smote.fit_sample(X_Train_Dtm, y_train)
print(X_sm_t.shape, y_sm_t.shape)

C:\Users\Admin34\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


(2682, 372998) (2682,)


In [38]:
SVM = LinearSVC()
SVM.fit(X_sm_t, y_sm_t)

LinearSVC()

In [39]:
for x in X_Test_Dtm:
    print(x)

  (0, 372327)	0.144725317883011
  (0, 371676)	0.05418860594627308
  (0, 365122)	0.06604513336807791
  (0, 359092)	0.08704333250179547
  (0, 355487)	0.06917286388078378
  (0, 353845)	0.08140127531150067
  (0, 338395)	0.10412755254601803
  (0, 337095)	0.09363370436457337
  (0, 314847)	0.144725317883011
  (0, 314843)	0.13741352590584358
  (0, 313388)	0.0446934037795739
  (0, 310501)	0.144725317883011
  (0, 310500)	0.144725317883011
  (0, 310417)	0.06308572132092681
  (0, 304511)	0.12213412066489354
  (0, 304503)	0.11398342869289313
  (0, 304399)	0.08496027730314289
  (0, 300992)	0.09472695410308839
  (0, 296636)	0.0727915781715862
  (0, 288187)	0.13222572693803034
  (0, 287706)	0.04838648146593888
  (0, 285383)	0.09311096927514331
  (0, 279966)	0.144725317883011
  (0, 279962)	0.13741352590584358
  (0, 278575)	0.0882110076916663
  :	:
  (0, 116094)	0.09917860604849114
  (0, 112486)	0.07119468609065437
  (0, 111947)	0.07074194855779461
  (0, 107116)	0.17194348828941441
  (0, 99776)	0.100682

In [40]:
X_Test_Dtm.shape

(750, 372998)

In [41]:
y_pred=SVM.predict(X_Test_Dtm)

In [42]:
metrics.accuracy_score(y_test,y_pred)

0.74

In [43]:
y_pred

array([3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 1, 1, 1, 3, 2, 3, 1, 2, 2, 3, 3, 1,
       2, 2, 1, 3, 1, 3, 1, 2, 1, 1, 3, 3, 2, 3, 1, 3, 3, 1, 1, 2, 1, 1,
       3, 1, 3, 3, 1, 1, 1, 1, 3, 2, 3, 3, 2, 3, 3, 1, 3, 1, 1, 3, 1, 2,
       3, 3, 2, 2, 3, 3, 3, 1, 3, 3, 1, 1, 2, 1, 1, 2, 3, 1, 3, 1, 3, 2,
       3, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 3, 2, 2, 1, 1, 3, 1, 3, 1, 1, 1,
       3, 1, 1, 3, 3, 1, 2, 3, 1, 3, 3, 3, 1, 1, 3, 2, 1, 3, 3, 3, 1, 3,
       2, 1, 1, 1, 3, 3, 1, 3, 1, 2, 1, 1, 3, 1, 3, 3, 3, 2, 3, 1, 3, 1,
       3, 3, 1, 1, 3, 3, 1, 3, 3, 2, 1, 3, 1, 1, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 2, 3, 3, 1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 1, 1, 1, 2, 3, 3, 1, 1,
       1, 2, 2, 2, 3, 1, 1, 3, 3, 3, 1, 2, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3,
       3, 1, 3, 3, 3, 3, 3, 1, 1, 3, 2, 3, 1, 1, 3, 3, 1, 2, 3, 3, 1, 3,
       1, 2, 1, 3, 3, 1, 3, 3, 1, 1, 1, 2, 1, 1, 1, 3, 3, 3, 3, 2, 1, 3,
       3, 1, 2, 3, 1, 3, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 1, 2,
       1, 1, 1, 3, 2, 3, 3, 1, 3, 1, 1, 1, 3, 1, 1,

In [44]:
print("Classification Report:\n\n", classification_report(y_test,y_pred))

Classification Report:

               precision    recall  f1-score   support

           1       0.76      0.81      0.78       278
           2       0.56      0.36      0.44       167
           3       0.78      0.88      0.83       305

    accuracy                           0.74       750
   macro avg       0.70      0.68      0.68       750
weighted avg       0.72      0.74      0.73       750



In [ ]:
## Accuracy report is 74% which can be increased by tuning the parameters ##